# Cloud sandbox powered by FIWARE

## About the open source GPL3 license and copyright for this product

Copyright © 2025 Computate Limited Liability Company in Utah, USA

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

ADDITIONAL TERMS

As stated in section 7. c) and e) of the GPL3 license, 
"you may supplement the terms of this License with terms," 
Computate has added the following additional terms to the license: 

  7 c) Prohibiting misrepresentation of the origin of that material, and
    requiring that modified versions of such material be marked in
    reasonable ways as different from the original version;

  7 e) Declining to grant rights under trademark law for use of some
    trade names, trademarks, or service marks;

Please do not redistribute this course until you have built your own platform with these tools, 
separate from the computate.org platform, and reconfigure your fork of this repo to deploy 
your own platform instead of the computate.org platform. 

QUESTIONS

For questions about this open source license, please contact our public mailing list at computate@group.computate.org


## Switch to the Python 3.9 Kernel

When opening this Jupyter Notebook in VSCode, you will need to switch to the Jupyter 3.9 Kernel. 

## Clone the smart-aquaculture site repository

We will be using the smart-aquaculture open source website, which has built-in support for Smart Aquaculture Smart Data Models. 

In [ ]:
%%bash
git clone https://github.com/computate-org/smart-aquaculture.git ~/smart-aquaculture
git clone https://github.com/computate-org/smart-aquaculture-static.git ~/smart-aquaculture-static
echo DONE

## Run SQL create scripts with new PostgreSQL tables
When we create new Smart Data Models that are persisted in the PostgreSQL database, we need to make sure that our database has all the tables and fields required to persist the data. Run the commands below to rsync the `db-create.sql` script to the PostgreSQL pod. It will connect again and apply all of the database schema changes to the database. 

In [ ]:
%%bash
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
oc exec pod/postgresql-0 -- mkdir -p /bitnami/postgresql/sql/
oc rsync $SITE_SRC/src/main/resources/sql/ \
  pod/postgresql-0:/bitnami/postgresql/sql/
oc exec pod/postgresql-0 -- \
  env PGPASSWORD=$(oc get secret/postgres-pguser-$DATABASE_USERNAME -o jsonpath={.data.password} | base64 -d) \
  psql -U $DATABASE_USERNAME -d $DATABASE_DATABASE -f /bitnami/postgresql/sql/db-create.sql
echo DONE

## Deploy site service

Create an OpenShift Service for internal networking to your site. 

In [ ]:
%%bash
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
cat <<EOF | oc apply -f -
apiVersion: v1
kind: Service
metadata:
  name: $SITE_SHORT_NAME
  labels:
    app.kubernetes.io/instance: $SITE_SHORT_NAME
    app.kubernetes.io/name: $SITE_SHORT_NAME
    deployment: $SITE_SHORT_NAME
    app: $SITE_SHORT_NAME
spec:
  ports:
    - name: http
      port: $SITE_PORT
      protocol: TCP
      targetPort: $SITE_PORT
    - name: cluster
      port: $CLUSTER_PORT
      protocol: TCP
      targetPort: $CLUSTER_PORT
  selector:
    app.kubernetes.io/instance: $SITE_SHORT_NAME
    app.kubernetes.io/name: $SITE_SHORT_NAME
    deployment: $SITE_SHORT_NAME
    app: $SITE_SHORT_NAME
  sessionAffinity: None
  type: ClusterIP
EOF
echo DONE

## Deploy site route

Create an OpenShift Route for external networking to your site. 

In [ ]:
%%bash
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
cat <<EOF | oc apply -f -
apiVersion: route.openshift.io/v1
kind: Route
metadata:
  name: $SITE_SHORT_NAME
  labels:
    app.kubernetes.io/instance: $SITE_SHORT_NAME
    app.kubernetes.io/name: $SITE_SHORT_NAME
    deployment: $SITE_SHORT_NAME
    app: $SITE_SHORT_NAME
spec:
  host: "$SITE_HOST_NAME"
  port:
    targetPort: http
  to:
    kind: Service
    name: "$SITE_SHORT_NAME"
    weight: 100
  tls:
    termination: edge
    insecureEdgeTerminationPolicy: Redirect
  wildcardPolicy: None
EOF
echo DONE

# Deploy your site! 

The next command will create a Deployment for your site. 

In [ ]:
%%bash
eval $(env SITE_NAMESPACE="$(oc project -q)" ./vars.py)
cat <<EOF | oc apply -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: $SITE_SHORT_NAME
  labels:
    app.kubernetes.io/instance: $SITE_SHORT_NAME
    app.kubernetes.io/name: $SITE_SHORT_NAME
    deployment: $SITE_SHORT_NAME
    app: $SITE_SHORT_NAME
spec:
  replicas: 1
  revisionHistoryLimit: 0
  strategy:
    type: RollingUpdate
  selector:
    matchLabels:
      app.kubernetes.io/instance: $SITE_SHORT_NAME
      app.kubernetes.io/name: $SITE_SHORT_NAME
      deployment: $SITE_SHORT_NAME
      app: $SITE_SHORT_NAME
  template:
    metadata:
      labels:
        app.kubernetes.io/instance: $SITE_SHORT_NAME
        app.kubernetes.io/name: $SITE_SHORT_NAME
        deployment: $SITE_SHORT_NAME
        app: $SITE_SHORT_NAME
    spec:
      containers:
        - name: $SITE_SHORT_NAME
          image: '$CONTAINER_URI'
          imagePullPolicy: Always
          ports:
            - containerPort: $SITE_PORT
              name: http
              protocol: TCP
          livenessProbe:
            initialDelaySeconds: 20
            periodSeconds: 10
            successThreshold: 1
            tcpSocket:
              port: 'http'
            timeoutSeconds: 10
            failureThreshold: 10
          readinessProbe:
            initialDelaySeconds: 20
            periodSeconds: 10
            successThreshold: 1
            tcpSocket:
              port: 'http'
            timeoutSeconds: 10
            failureThreshold: 10
          resources:
            requests:
              cpu: '100m'
              memory: '100Mi'
            limits:
              cpu: '500m'
              memory: '500Mi'
          envFrom:
            - secretRef:
                name: $SITE_SHORT_NAME-zookeeper
            - secretRef:
                name: $SITE_SHORT_NAME-solr
            - secretRef:
                name: $SITE_SHORT_NAME-database
            - secretRef:
                name: $SITE_SHORT_NAME-auth
            - secretRef:
                name: $SITE_SHORT_NAME-rabbitmq
            - secretRef:
                name: $SITE_SHORT_NAME-context-broker
            - secretRef:
                name: $SITE_SHORT_NAME-site
EOF
echo DONE

### View site pod details
After deploying the site, it will take a minute before the site pod is up and running. Run the command below until the site pod health checks are `READY 1/1` and `STATUS Running`. 

In [ ]:
%%bash
oc get pod -l app=smartaquaculture
oc wait pod -l app=smartaquaculture --for=condition=Ready --timeout=2m
oc get pod -l app=smartaquaculture
echo DONE

### View Keycloak pod logs
If your Keycloak pod does not reach the STATUS Running, you can run the command below to view the pod logs of Keycloak and check for other errors that may have occured. 

In [ ]:
%%bash
oc logs -l app=smartaquaculture
echo DONE

# Create a Keycloak user

Access the Keycloak dashboard using the route. 

In [ ]:
%%bash
echo "https://$(oc get $(oc get route -l app.kubernetes.io/name=keycloak -o name) -o jsonpath={.spec.host})"
echo DONE

The username will be `admin` and run the command below to find the admin password.

In [ ]:
%%bash
oc get secret/keycloak-initial-admin -o jsonpath={.data.password} | base64 -d
echo DONE

- At the top left, switch the realm from `Keycloak` to `COMPUTATE.ORG`. 
- On the left, click on `Users`. 
- Set `Email verified`. 
- Type in a username, email, first name, and last name. 
- Click `Create`. 

## Add user to SuperAdmin group

- While still on the user page, click `Groups`. 
- Click `Join Group`. 
- Select `SuperAdmin`. 
- Click `Join`. 

## Set a user password

- 

# Access the site

Access the site using the route. 

In [ ]:
%%bash
echo "https://$(oc get route/computateorg -o jsonpath={.spec.host})"
echo DONE